In [5]:
from IPython.core.display import Markdown, display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import traceback
import numpy as np
import scipy.stats as ss
import yaml
import itertools
import sys
import os
import subprocess
from os import path
import pickle
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

from sklearn.linear_model import LinearRegression, TheilSenRegressor, RANSACRegressor, HuberRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline

def printmd(*args):
    display(Markdown(' '.join(map(str, args))))

In [6]:
!ros2 run local_planning_performance_modelling compute_metrics -r "~/tmp/test_collision_metric/*" -s

/bin/bash: ros2: command not found


In [7]:
df = pd.read_csv(path.expanduser("~/tmp/test_collision_metric/results.csv"))
with open(path.expanduser("~/tmp/test_collision_metric/results_info.yaml")) as results_info_file:
    results_info = yaml.safe_load(results_info_file)

# turn odometry_error into alpha_1..4
results_info['run_parameter_names'] += ['alpha_1', 'alpha_2', 'alpha_3', 'alpha_4']
results_info['run_parameter_names'].remove('odometry_error')
for i in range(0, 4):
    df[f'alpha_{i+1}'] = df['odometry_error'].apply(lambda x: eval(x)[i])
del df['odometry_error']

df.loc[df.robot_model == 'turtlebot3_waffle_performance_modelling', 'robot_model'] = 'turtle'
df.loc[df.robot_model == 'hunter2', 'robot_model'] = 'hunter'

run_parameters = [c for c in list(df.columns) if c in results_info['run_parameter_names']]
metrics_versions = [c for c in list(df.columns) if '_version' in c]
everything_else = ['run_id', 'session_id', 'run_number']
metrics = [c for c in df.columns if c not in metrics_versions + run_parameters + everything_else]
metrics_and_versions = [c for c in list(df.columns) if '_version' in c or c in metrics]

cpu_time_metrics = [c for c in metrics if 'cpu_time' in c]
max_memory_metrics = [c for c in metrics if 'max_memory' in c]
odometry_metrics = [c for c in metrics if 'odometry_error_alpha_' in c]
odometry_mean_metrics = [c for c in metrics if 'odometry_error_alpha_' in c and '_mean' in c]
odometry_std_metrics = [c for c in metrics if 'odometry_error_alpha_' in c and '_std' in c]
odometry_parameters = [f'alpha_{i}' for i in range(1, 5)]
localization_update_metrics = [c for c in metrics if 'localization_update_' in c]
localization_update_mean_metrics = [c for c in metrics if 'localization_update_' in c and '_mean' in c]
localization_update_translation_mean_metrics = [c for c in metrics if 'localization_update_' in c and '_translation_' in c and '_mean' in c]
navigation_metrics = [c for c in metrics if c in ['trajectory_length', 'execution_time', 'success_rate']]

# add metrics from existing ones
df['mean_velocity'] = df['trajectory_length'] / df['execution_time']
navigation_metrics += ['mean_velocity']
metrics += ['mean_velocity']
metrics_and_versions += ['mean_velocity']

# add useful parameters
df['session_id'] =  df['run_id'].apply(lambda x:  x.split('_')[1]+'_'+x.split('_')[2]+'_'+x.split('_')[3])
df['run_number'] =  df['run_id'].apply(lambda x:  int(x.split('_')[5]))
df[max_memory_metrics] = df[max_memory_metrics]/1024**2
df.max_steering_angle_deg = df.max_steering_angle_deg.fillna(90)

printmd("## Number of Runs")
printmd(f"    {len(df.run_id.unique())}")

printmd("## Run Parameters")
for name in [run_parameter_name for run_parameter_name in run_parameters if 'localization_generator_' not in run_parameter_name]:
    values = list(df[name].unique())
    printmd(f"    {name:<70}", sorted(values))

printmd("## Metrics")
for name in metrics_and_versions:
    if name in metrics_versions:
        if len(df[name].unique()) == 1:
            printmd(f"    {name:<70} {sorted(df[name].unique())}")
        else:
            printmd(f"<code><font style='background-color:yellow;font-family:monospace'>{name:<70}{sorted(df[name].unique())} </font></code> ⚠️")
    else:
        printmd(f"    {name:<70} min: {df[name].min(skipna=True):10.4g} {'avg':>15}: {df[name].mean(skipna=True):10.4g} {'max':>15}: {df[name].max(skipna=True):10.4g}]")

## Number of Runs

    6

## Run Parameters

    environment_name                                                       ['7A-2', 'airlab']

    global_planner_node                                                    ['navfn', 'smac']

...

    average_rotation                                                       min:        nan             avg:        nan             max:        nan]

    motion_characteristics_version                                         [nan]

    average_translation_velocity                                           min:    0.08879             avg:     0.1951             max:     0.2569]

    average_rotation_velocity                                              min:    0.05083             avg:      0.153             max:      0.266]

    mean_velocity                                                          min:          0             avg:     0.1327             max:     0.2418]

In [8]:
df[['run_id', 'robot_model', 'collision_rate']]

,run_id,robot_model,collision_rate
0,session_2022-01-12_08-36-17_112051_run_000000000,turtle,0
1,session_2022-02-08_16-22-57_112159_run_000000000,turtle,0
2,session_2022-02-08_16-36-21_171560_run_000000000,turtle,1
3,session_2022-02-22_09-29-56_465082_run_000000000,hunter,1
4,session_2022-02-22_09-35-14_434687_run_000000000,hunter,1
5,session_2022-02-22_09-45-01_947183_run_000000000,hunter,0
